In [1]:
from scipy.stats import spearmanr

In [2]:
r1 = [1, 2, 3, 4, 5]
r2 = [2, 3, 1, 2, 3]

In [3]:
spearmanr(r1, r2)

SpearmanrResult(correlation=0.21081851067789192, pvalue=0.7335793204938506)

In [ ]:
# generate rank
if not os.listdir('rank-result'):
    from rank_utility import *
    # - click rank
    click_rank_files = [f for f in os.listdir(data_path) if "click_rank" in f]
    generate_click_rank(data_path, click_rank_files)
    # - general rank
    pairwise_rank_files = [f for f in os.listdir(data_path) if "pairwise_rank" in f]
    generate_pairwise_rank(data_path, pairwise_rank_files)